In [105]:
#import Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [114]:
# The code was removed by Watson Studio for sharing.

,province,household,telephone,pc,notebook,tablet
0,Bangkok,2924500.0,1194298.0,1076639.0,1200643.0,950754.0
1,Samut Prakan,724752.0,119065.0,231834.0,103538.0,98020.0
2,Nonthaburi,538712.0,176176.0,161888.0,165580.0,174418.0
3,Pathum Thani,502671.0,91839.0,122222.0,128676.0,117960.0
4,Phranakhon Si Ayutthaya,257462.0,14906.0,45823.0,56127.0,40410.0


In [115]:
#Convert Number to Rate per household

data['telephone'] = data['telephone']/data['household']
data['pc'] = data['pc']/data['household']
data['notebook'] = data['notebook']/data['household']
data['tablet'] = data['tablet']/data['household']

data.head()

,province,household,telephone,pc,notebook,tablet
0,Bangkok,2924500.0,0.408377,0.368145,0.410546,0.325100
1,Samut Prakan,724752.0,0.164284,0.319880,0.142860,0.135246
2,Nonthaburi,538712.0,0.327032,0.300509,0.307363,0.323769
3,Pathum Thani,502671.0,0.182702,0.243145,0.255985,0.234666
4,Phranakhon Si Ayutthaya,257462.0,0.057896,0.177980,0.218001,0.156955


In [116]:
#Load geodata
body = client_0238d54ef7f74cb2b06d4b8df566b7a3.get_object(Bucket='ibmdatasciencecapstoneproject-donotdelete-pr-l2dsu5njhtxa86',Key='th.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo_data = pd.read_csv(body)


# Renaming the columns
geo_data.columns = ['province', 'latitude', 'longitude']
geo_data.head()


,province,latitude,longitude
0,Bangkok,13.730579,100.523884
1,Chiang Mai,18.790376,98.984683
2,Samut Prakan,13.600417,100.596778
3,Nakhon Ratchasima,14.971596,102.082550
4,Hat Yai,7.008359,100.476682


In [117]:
#Merging dataframes
merged_data = pd.merge(data, geo_data, on='province')
merged_data['coordinates'] = list(zip(merged_data['latitude'], merged_data['longitude']))
merged_data['coordinates'] = merged_data['coordinates'].apply(Point)

gdf = gpd.GeoDataFrame(merged_data, geometry='coordinates')
gdf.head()

,province,household,telephone,pc,notebook,tablet,latitude,longitude,coordinates
0,Bangkok,2924500.0,0.408377,0.368145,0.410546,0.325100,13.730579,100.523884,POINT (13.731 100.524)
1,Samut Prakan,724752.0,0.164284,0.319880,0.142860,0.135246,13.600417,100.596778,POINT (13.600 100.597)
2,Nonthaburi,538712.0,0.327032,0.300509,0.307363,0.323769,13.862167,100.513417,POINT (13.862 100.513)
3,Pathum Thani,502671.0,0.182702,0.243145,0.255985,0.234666,14.013461,100.530488,POINT (14.013 100.530)
4,Ang Thong,82666.0,0.085101,0.180280,0.172163,0.089650,14.588389,100.452834,POINT (14.588 100.453)


In [118]:
# import map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON 
import geopy

# use geopy to obtain latitude/longitude of Toronto.
# coordinates will be used for map visualization

address = 'Thailand'

geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Thailand {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Thailand 14.8971921, 100.83273.


In [119]:
import folium

# create map of Toronto using latitude and longitude values
map_thailand = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, neighbourhood in zip(gdf['latitude'], gdf['longitude'], gdf['province']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_thailand)  
    
map_thailand

# Cluster Analysis

In [120]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [122]:
n_group = 6 # we will group neighbourhoods into 6 clusters
data_temp = data.drop("province", axis=1)
data_temp = data_temp.fillna(0)

# run k-means clustering
kmeans = KMeans(n_clusters=n_group, random_state=0).fit(data_temp)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
data.insert(1, 'label', kmeans.labels_)

In [123]:
data_temp.head()


,household,telephone,pc,notebook,tablet
0,2924500.0,0.408377,0.368145,0.410546,0.325100
1,724752.0,0.164284,0.319880,0.142860,0.135246
2,538712.0,0.327032,0.300509,0.307363,0.323769
3,502671.0,0.182702,0.243145,0.255985,0.234666
4,257462.0,0.057896,0.177980,0.218001,0.156955


In [124]:
data_cluster = pd.merge(data, geo_data, on='province')
data_cluster.head()

,province,label,household,telephone,pc,notebook,tablet,latitude,longitude
0,Bangkok,1,2924500.0,0.408377,0.368145,0.410546,0.325100,13.730579,100.523884
1,Samut Prakan,3,724752.0,0.164284,0.319880,0.142860,0.135246,13.600417,100.596778
2,Nonthaburi,4,538712.0,0.327032,0.300509,0.307363,0.323769,13.862167,100.513417
3,Pathum Thani,4,502671.0,0.182702,0.243145,0.255985,0.234666,14.013461,100.530488
4,Ang Thong,5,82666.0,0.085101,0.180280,0.172163,0.089650,14.588389,100.452834


In [125]:
from matplotlib import cm, colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_group)
ys = [i + x + (i*x)**2 for i in range(n_group)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_cluster['latitude'], data_cluster['longitude'], data_cluster['province'], data_cluster['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters